In [ ]:
import pandas as pd
import copy
import csv
import re

In [ ]:
def printAllKLength(set, k):   
    n = len(set)
    printAllKLengthRec(set, "", n, k)
    

def printAllKLengthRec(set, prefix, n, k): 
    if (k == 0) :
        res.append(prefix)
        return
  
    for i in range(n):
        newPrefix = prefix + set[i]
        printAllKLengthRec(set, newPrefix, n, k - 1)

combs = []
res = []
for i in range(7):
    printAllKLength(['A', 'C', 'G', 'T'], i+1)
    combs.append(res)
    res = copy.deepcopy([])

In [ ]:
def kmerize(file, k, is_set, data, filename, removed=False):
    if removed:
        csv_out = open(filename + '.Removed.k' + str(k) + '.csv', 'w', newline='')
    else:
        csv_out = open(filename + '.Cleaned.k' + str(k) + '.csv', 'w', newline='')

    writer = csv.writer(csv_out, delimiter=',')
    hasHeader = False
    a = 0
    with open(file) as f:
        for line in f:
            feature_dict = {}
            tokens = line.split(',')
            seq = tokens[1].strip()
            genus = tokens[2].strip()
            genus = re.findall('[a-zA-Z]+', genus)[0]
            if len(tokens) > 4:
                class_ = tokens[3].strip()
            if a == 0:
                a = 1
                continue
            # skip sequences with len < 657 (see p. 7 and ref [30] in the paper)
            if len(seq) < 657:    # change to len(seq) >= 657 to get removed data
                continue

            for i in range(len(seq)-k+1):
                kmer = seq[i:i+k]
                check = False
                for nucleotide in kmer:
                    if nucleotide not in ['A', 'C', 'G', 'T']:
                        check = True
                        break
                if check: continue
                feature_dict.setdefault(kmer, 0)
                feature_dict[kmer] += 1
            
            for key in data:
                if key not in feature_dict:
                    feature_dict[key] = 0
                    
            if not hasHeader:
                sorted_features = list(feature_dict.keys())
                sorted_features.sort()
                writer.writerow(['', 'nucleotide', 'genus name'] + sorted_features)
                hasHeader = True
                
            feature_dict = dict(sorted(feature_dict.items()))
            counter_list = list(feature_dict.values())
            writer.writerow(['', seq, genus] + counter_list)
    csv_out.close()

files = ['Pucciniomycetes.csv', 'Polypodiopsida.csv', 'Aves.csv', 'Chiroptera.csv', 'Rodentia.csv']

for k in range(7):
    for file in files:
        kmerize('../raw_data/'+file, k+1, False, combs[k], file.split('.')[0])